# Módulo 8: Serotipado *Streptococcus pneumoniae* y *Streptococcus agalactiae* 

## Descripción general

Este módulo será desarrollado en dos partes:

1. Parte 1: Prediciendo serotipos de *S. pneumoniae*

2. Parte 2: Prediciendo serotipos de *S. agalactiae*

### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

> **Nota**: En este módulo utilizaremos SeroBA para la predicción de serotipos de *S. pneumoniae*. Sin embargo, el repositorio original de esta herramienta no tiene soporte actualmente. Por lo tanto, utilizaremos el repositorio de [Bentley-group](https://github.com/sanger-bentley-group/seroba) donde se recomienta el uso de la herramienta con Docker.

In [ ]:
# Instala udocker
%%shell
pip install udocker
udocker --allow-root install

In [ ]:
# Obtener SeroBA
!udocker --allow-root pull sangerbentleygroup/seroba

In [ ]:
# Compruebe si se seroBA se instaló correctamente
!udocker --allow-root run sangerbentleygroup/seroba seroba version

>**Nota**: En este módulo utilizaremos srst2 para la predicción de serotipos de *S. agalactiae*. Sin embargo, esta herramienta no funciona adecuadamente en Python3. Por lo tanto, instalaremos Python2 y ejecutaremos la herramienta utilizando la versión python=2.7 con los comandos `!conda run -n py2_env`

In [ ]:
# Crear un ambiente en conda con la version 2.7 de python. El ambiente se llamará python2 
!conda create -n py2_env python=2.7
#!apt-get install python2
!conda run -n py2_env python --version

In [ ]:
#Instalar srst2
!conda run -n py2_env conda install -c bioconda srst2 --yes

In [ ]:
# Compruebe si se srst2 se instaló correctamente
!conda run -n py2_env srst2 --help

### Descargar datos

In [ ]:
!wget https://zenodo.org/records/13750987/files/Module_8.tar.gz

In [ ]:
!tar xvf Module_8.tar.gz

## Parte 1: Prediciendo serotipos de *S. pneumoniae*

Hasta la fecha, hay más de 100 serotipos conocidos descritos para *S. pneumoniae* según las diferentes propiedades bioquímicas y antigénicas de la cápsula. Hay una serie de métodos in-silico para detectar el locus cps, que luego se pueden utilizar para predecir serotipos a partir de datos WGS. La identificación precisa de los serotipos neumocócicos es importante para rastrear la distribución y evolución de los serotipos luego de la introducción de vacunas efectivas.

Se desarrolló [SeroBA](https://github.com/sanger-pathogens/seroba?tab=readme-ov-file#installation), el cual hace un uso eficiente de los recursos computacionales además de detectar con precisión el locus cps en baja cobertura y predice serotipos a partir de datos WGS utilizando una base de datos adaptada de PneumoCaT. SeroBA puede predecir serotipos mediante la identificación del locus cps, directamente a partir de datos de lectura de secuenciación del genoma completo sin procesar con un 98 % de concordancia utilizando un método basado en k-mer, puede procesar 10 000 muestras en poco más de 1 día utilizando un servidor estándar y puede llamar serotipos en una cobertura tan baja como 10x. SeroBA está implementado en Python3 y está disponible gratuitamente bajo una licencia GPLv3 de código abierto.

*Lectura adicional*: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6113868/

Explore el uso de serobar ejecutando ``seroba -h``

In [ ]:
# Ejecutar seroBA
!udocker --allow-root run sangerbentleygroup/seroba seroba --help

#### SeroBA requiere solo tres archivos de entrada:

1. Base de datos con kmc (utilidad diseñada para contar k-mers) y ariba (identificación de resistencia antimicrobiana por ensamblaje).

Para este módulo usaremos la base de datos `seroba_k71_14082017` que se encuentra en la carpeta del módulo.

 Si desea crear la base de datos ud mismo en su ordenador, puede seguir los pasos que se encuentran [aquí](https://github.com/sanger-pathogens/seroba?tab=readme-ov-file#setting-up-the-database).

2. Archivos de secuencias forward y reverse en fastq.

Utilizaremos los archivos pareados ERR331173_1.fastq.gz y ERR331173_2.fastq.gz los cuales corresponden al run accession ERR331173 del proyecto [PRJEB3084](https://www.ebi.ac.uk/ena/browser/view/PRJEB3084).

Algunos datos importantes de la muestra:

- País de origen: Perú
- Organismo: *Streptococcus pneumoniae*
- Instrumento: ILLUMINA
- Modelo del instrumento: Illumina MiSeq 
- Conteo de reads: 1203646
- Conteo de bases: 240729200 
- Nombre del centro: Wellcome Sanger Institute;SC 
- Diseño de librerías:  PAIRED
- Estrategia de librerías: WGS

3. Prefijo de salida.

### Para predecir el serotipo de una sola cepa , usaremos el comando:

In [ ]:
# Correr Seroba con udocker
!udocker --allow-root run -v /content/Module_8/seroba_k71_14082017:/seroba_k71_14082017 -v /content/Module_8:/fastq_files sangerbentleygroup/seroba seroba runSerotyping /seroba_k71_14082017 /fastq_files/ERR1795461_1.fastq.gz /fastq_files/ERR1795461_2.fastq.gz /fastq_files/output_test

Una explicación de este comando es la siguiente:

`dockerrun` es una función para iniciar un contenedor

`sangerbentleygroup/seroba` es la imagen acoplable [sangerbentleygroup - representa el repositorio y seroba - representa la imagen del contenedor]

`seroba` es la herramienta/programa

`runSerotyping` especifica que el programa realizará la serotipificación

`seroba_k71_14082017` especifica dónde está el directorio de seroba

`ERR331173_1.fastq.gz` y `ERR331173_2.fastq.gz` son los archivos fastq de avance y retroceso

`ERR331173_output` especifica el prefijo de salida

En la carpeta de salida, encontrará un **pred.tsv** que incluye su serotipo predicho.

### Predecir el serotipo de múltiples cepas

>**Nota:** En este módulo no ejecutaremos el serotipado múltiple por la falta de recursos en Colab. Sin embargo, aquí tienes un ejemplo de como hacerlo.

### Primero crearemos una carpeta para cada par de archivos FASTQ comprimidos y el nombre de la identificación de la cepa usando el comando:

In [ ]:
# No ejecutar
# Mover archivos
# for x in *1.fastq.gz; do mkdir ${x%%_1.fastq.gz} ; mv $x ${x%%_1.fastq.gz}; mv ${x%%1.fastq.gz}2.fastq.gz ${x%%_1.fastq.gz}; done

Una explicación de este comando es la siguiente:

`for x en 1.fastq.gz;` Esto inicia un ciclo for que itera sobre todos los archivos en el directorio actual que terminan con "1.fastq.gz".

`do` Esto inicia el bloque de código que se ejecutará para cada archivo.

`mkdir ${x%%_1.fastq.gz}` Esto crea un nuevo directorio con el mismo nombre que el archivo, pero con el sufijo "_1.fastq.gz" eliminado.

`mv $x ${x%%_1.fastq.gz}` Esto mueve el archivo original al nuevo directorio creado en el paso anterior.

`mv ${x%%1.fastq.gz}2.fastq.gz ${x%%_1.fastq.gz}` Esto mueve un segundo archivo que tiene el mismo prefijo que el primer archivo, pero con un "2.fastq sufijo .gz", en el nuevo directorio creado en el primer paso.

`done` Esto finaliza el bucle for.

En general, este script está diseñado para tomar datos de secuenciación pareados que se almacenan en dos archivos separados con nombres que terminan en "_1.fastq.gz" y "_2.fastq.gz", y organizarlos en directorios según el prefijo de los nombres de archivo.

*Lectura adicional sobre los comandos de bucle*: https://www.gnu.org/software/bash/manual/bash.html#Looping-Constructs

### Después ejecutaremos seroBA usando el comando:

In [ ]:
# No ejecutar
# Correr seroBA en un loop
#for x in * ; do seroba runSerotyping seroba_k71_14082017 $x/${x}_1.fastq.gz $x/${x}_2.fastq.gz $x"_output"; done

Una explicación de este comando es la siguiente:

`for x in *;` Esto inicia un ciclo for que itera sobre todos los archivos en el directorio actual.
`do` Esto inicia el bloque de código que se ejecutará para cada archivo.

`seroba runSerotyping seroba_k71_14082017` Esto ejecuta el comando "runSerotyping" dentro del contenedor Docker, usando la base de datos "seroba_k71_14082017".

`$x/${x}_1.fastq.gz $x/${x}_2.fastq.gz` Estos son los archivos de entrada para el comando, ubicados en un directorio con el nombre del archivo (sin el carácter "#") y con "_1.fastq.gz" o "_2.fastq.gz" añadido al nombre.

`$x"_output"` Este es el directorio de salida para el comando, también nombrado después del archivo (con "_output" añadido al nombre).

`;` Esto separa el comando Docker del final del bucle for.

`done` Esto finaliza el bucle for.

### Finalmente compilaremos los resultados de las ejecuciones anteriores con el comando:

Este comando combinará las salidas de seroba en un archivo tsv.

In [ ]:
# No ejecutar
# Resumen de los resultados
#!seroba summary ./

___

## Parte 2: Prediciendo serotipos de *S. agalactiae*

*Streptococcus agalactiae* (*Streptococcus* del grupo B o GBS) se divide actualmente en diez serotipos según los antígenos capsulares específicos del tipo y se denominan Ia, Ib, II, III, IV, V, VI, VII, VIII y IX.

Group B Streptococcus Serotyping by Genome Sequencing repository contiene un archivo de referencia seleccionado que se puede utilizar para serotipificar *Streptococcus agalactiae* *in silico* con datos de secuenciación del genoma completo. El archivo de referencia (GBS-SBG.fasta) está diseñado para que se pueda utilizar tanto para el mapeo de lectura corta como para las estrategias basadas en ensamblaje.

*Lectura adicional*: https://github.com/swainechen/GBS-SBG


El programa [SRST2](https://github.com/katholt/srst2 ) (Short Read Sequence Typing for Bacterial Pathogens) está diseñado para tomar datos de secuencias de Illumina, una base de datos MLST y/o una base de datos de secuencias de genes (por ejemplo, genes de resistencia, genes de virulencia, etc.) e informar la presencia de ST y/o genes de referencia. Este programa realiza una detección rápida y precisa de genes y alelos a partir de lecturas de secuenciación corta WGS. SRST2 puede tipicar lecturas utilizando cualquier base de datos de secuencias y puede calcular tipos de secuencias combinatorias definidas en bases de datos tipo MLST.

*Lectura adicional*: https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-014-0090-6

SRST2 requiere:

1. lecturas pareadas: 

En esta sección analizaremos los datos pertenecientes al estudio [Near-term pregnant women in the Dominican Republic experience high rates of Group B Streptococcus rectovaginal colonization with virulent strains](https://doi.org/10.1371/journal.pgph.0002281) realizado por Laycock KM, Acosta F, Valera S, Villegas A, Mejia E, Mateo C, et al. (2023). Utilizaremos los archivos pareados FASTQ SRR23874884_1.fastq.gz y SRR23874884_2.fastq.gz los cuales corresponden al run accession SRR23874884 del proyecto [PRJNA945321](https://www.ebi.ac.uk/ena/browser/view/PRJNA945321)	.

Algunos datos importantes de la muestra:

- País de origen: República Dominicana
- Organismo: *Streptococcus agalactiae*
- Instrumento: ILLUMINA
- Modelo del instrumento: Illumina NovaSeq 6000  
- Conteo de reads: 1916182
- Conteo de bases: 578686964 
- Nombre del centro: SUB12954699  
- Diseño de librerías:  PAIRED
- Estrategia de librerías: WGS

2. Una base de datos de secuencias en un archivo fasta:

GBS-SBG.fasta

### Para predecir el serotipo de una sola cepa usaremos el comando:

In [ ]:
# Primero nos dirigimos a la carpeta que contiene los datos de la Parte 2 del módulo
%cd /content/Module_8/s_agalactiae/

In [ ]:
# Correr srst2
!conda run -n py2_env srst2 --input_pe SRR23874884_1.fastq.gz SRR23874884_2.fastq.gz --output SRR23874884 --log --gene_db GBS-SBG.fasta

Una explicación de este comando es la siguiente:

`conda` Es la herramienta de línea de comandos para gestionar entornos y paquetes en Conda, que es un sistema de gestión de paquetes y entornos.

`run` Este subcomando de Conda se usa para ejecutar comandos dentro de un entorno Conda específico sin necesidad de activar el entorno primero.

`-n py2_env`: La opción -n especifica el nombre del entorno Conda en el que se deben ejecutar los comandos. En este caso, py2_env es el nombre del entorno que hemos creado.

`srst2` es la herramienta

`--input_pe` especifica que el archivo de entrada son lecturas finales emparejadas que son SRR23874884_1.fastq.gz y SRR23874884_2.fastq.gz 

`--output `especifica el archivo de salida SRR23874884

`--log` active el registro en el archivo, en lugar de la salida estándar

`--gene_db` especifica la base de datos GBS-SBG.fasta

Ejecute el comando `ls -lh` para verificar el contenido de la carpeta.

In [ ]:
# Listar archivos
!ls -lh

El archivo de salida de la ejecución anterior es `SRR23874884__genes__GBS-SBG__results.txt`

Entonces, ejecutaremos `cat` `SRR23874884__genes__GBS-SBG__results.txt` para ver el contenido de este archivo

In [ ]:
# Ver el contenido de un archivo
!cat SRR23874884__genes__GBS-SBG__results.txt

### Para ejecutar SRST2 en múltiples cepas, ejecute el comando:

>**Nota:** En este módulo no ejecutaremos SRST2 múltiple por la falta de recursos en Colab. Sin embargo, aquí tiene un ejemplo de como hacerlo.

In [ ]:
# No ejecutar
# Correr srst2 en un loop
#!conda run -n py2_env srst2 --input_pe *.fastq.gz --output s.agalactiae --log --gene_db GBS-SBG.fasta

`--input_pe *.fastq.gz` especifica que el archivo de entrada son varios archivos fastq.gz comprimidos.

## BONO!

Si está trabajando con BASH en su computadora o en un HPC y tiene demasiados archivos, puede optimizar los comandos, los bucles (for loops) son muy útiles para grandes conjuntos de datos.

Aquí hay una manera de hacerlo.

Cree un nuevo script bash usando nano llamado `serotype.sh`

In [ ]:
#!/bin/bash
#Nombre de archivo: Serotype.sh
#Este script para serotipificar S.pneumoniae a partir de fastq.gz recortados

function docker_run() { docker run --rm=True -u $(id -u):$(id -g) -v $(pwd):/data "$@" ;}
wordir=/path/to/your/directory/
cd $wordir

for i in $(ls *_1.trimmed.fastq.gz); do

NAME=$(basename $i _1.trimmed.fastq.gz)
echo "$NAME"
j="${NAME}_1.trimmed.fastq.gz"
echo "$j"
k="${NAME}_2.trimmed.fastq.gz"
echo "$k"

docker_run staphb/seroba seroba runSerotyping seroba_k71_14082017 ./$j ./$k ${NAME}_serotype_output;

done

Movemos todas las salidas a una nueva carpeta  `mv *_serotype_output serotype_results`, luego compilamos todos los datos dentro del nuevo directorio con el comando `seroba summary ./`  debe obtener un archivo tsv.

Cree un nuevo script bash usando nano llamado `serotype_2.sh`

In [ ]:
#!/bin/bash
#Este script para serotipificar múltiples lecturas genómicas de S. agalactaeae

function docker_run() { docker run --rm=True -u $(id -u):$(id -g) -v $(pwd):/data "$@" ;}
wordir=/path/to/your/directory/
cd $wordir
mkdir -p serotyping_output

for i in $(ls *_1.fastq.gz); do

NAME=$(basename $i _1.fastq.gz)
echo "$NAME"
j="${NAME}_1.fastq.gz"
echo "$j"
k="${NAME}_2.fastq.gz"
echo "$k"

docker_run staphb/srst2 srst2 --input_pe $j $k --output ./serotyping_out/${NAME}_output --log --gene_db analysis/clean_data/GBS-SBG.fasta;

done

*Adaptado de:*

- Advanced Bioinformatics Course developed for the GPS and JUNO projects - Wellcome Sanger Insitute
    
*Modificado por Luisa Sacristán (Universidad de los Andes-CABANA)*